# Types and Dispatch in Julia

Julia is built around types.

Software architectures in Julia are built around good use of the type system.

# Abstract vs concrete types

*Concrete types* are the types of objects. They specify the data structure of an object.

*Abstract types* cannot be instantiated. They define sets of related concrete types (their descendants) by their behavior.

In [2]:
typeof(3)

Int64

In [3]:
typeof(3.0)

Float64

In [6]:
isconcretetype(Float64)

true

In [7]:
isabstracttype(Number)

true

In [8]:
isabstracttype(Real)

true

### Duck typing

A `Number` is some abstract type that can do things like `+`,`-`,`*`, and `/`. In this category we have (concrete) things like `Float64` and `Int32`.

An `AbstractArray` is a type that can be indexed like `A[i]`. An `AbstractArray` may be mutable, meaning it can be set: `A[i]=v`.

### Inspecting the type tree

In [9]:
supertype(Float64)

AbstractFloat

In [10]:
supertype(AbstractFloat)

Real

In [12]:
subtypes(AbstractFloat)

4-element Array{Any,1}:
 BigFloat
 Float16 
 Float32 
 Float64 

In [13]:
supertype(Real)

Number

In [14]:
supertype(Number)

Any

Everything is a subtype of `Any`

In [15]:
Number <: Any

true

In [16]:
Float64 <: Any

true

In [17]:
Int32 <: Any

true

In [18]:
Int32 <: String

false

There is also `isa` for objects:

In [20]:
3.0 isa Float64

true

In [21]:
3 isa Float64

false

In [22]:
Int32(3) isa Int32

true

In [24]:
typeof(3) <: Float64

false

We define a function that, given a concrete type `T`, prints the single branch of the type tree that leads from the top node `Any` to the leave `T`.

In [25]:
function show_supertypes(T) 
 print(T)
 while T != Any 
     T = supertype(T) 
     print(" <: ", T) 
 end 
end

show_supertypes (generic function with 1 method)

In [26]:
show_supertypes(Float64)

Float64 <: AbstractFloat <: Real <: Number <: Any

In [27]:
show_supertypes(String)

String <: AbstractString <: Any

Let's extract a bunch of branches

In [29]:
function show_subtypetree(T, level=1, indent=4)
   level == 1 && println(T)
   for s in subtypes(T)
     println(join(fill(" ", level * indent)) * string(s))
     show_subtypetree(s, level+1, indent)
   end
end

show_subtypetree (generic function with 3 methods)

In [30]:
show_subtypetree(Number)

Number
    Complex
    Real
        AbstractFloat
            BigFloat
            Float16
            Float32
            Float64
        AbstractIrrational
            Irrational
        Integer
            Bool
            Signed
                BigInt
                Int128
                Int16
                Int32
                Int64
                Int8
            Unsigned
                UInt128
                UInt16
                UInt32
                UInt64
                UInt8
        Rational


Note that concrete types are the leaves of the type tree.

Abstract types are nodes in the type graph.

# Functions, Methods, and Dispatch

Let's define a *function* that calculates the absolute value of a number (like Julias `abs` already does).

How would we practically calculate the absolute values of the numbers $-4.32$ and $1.0 + 1.0i$?

Presumably:
* Real number: "Drop the sign." => `myabs(-4.32) = 4.32`
* Complex number: "Square root of z times the complex conjugate of z." => `myabs(1.0 + 1.0im) = sqrt(2) ≈ 1.414`

We see that the *methods* that we use depend on the type of the number.

While the single **function** represents the *what* ("calculate the absolute value"), there might be different **methods** describing the *how*.

We can use the `::` operator to annotate function arguments with types and define different methods.

In [32]:
myabs(x::Float64) = sign(x) * x

myabs (generic function with 1 method)

In [33]:
myabs(-4.32)

4.32

In [34]:
myabs(1.0 + 1.0im)

MethodError: MethodError: no method matching myabs(::Complex{Float64})
Closest candidates are:
  myabs(!Matched::Float64) at In[32]:1

In [35]:
myabsthatdoesntexist(1.0 + 1.0im)

UndefVarError: UndefVarError: myabsthatdoesntexist not defined

In [36]:
myabs(z::ComplexF64) = sqrt(real(z * conj(z)))

myabs (generic function with 2 methods)

In [37]:
myabs(1.0 + 1.0im)

1.4142135623730951

In [38]:
methods(myabs)

# 2 methods for generic function "myabs":
[1] myabs(z::Complex{Float64}) in Main at In[36]:1
[2] myabs(x::Float64) in Main at In[32]:1

One can check which particular method is being used through the `@which` macro.

In [39]:
@which myabs(-4.32)

myabs(x::Float64) in Main at In[32]:1

In [40]:
@which myabs(1.0 + 1.0im)

myabs(z::Complex{Float64}) in Main at In[36]:1

Note that we should better loosen our type restrictions:

In [41]:
myabs(-3)

MethodError: MethodError: no method matching myabs(::Int64)
Closest candidates are:
  myabs(!Matched::Complex{Float64}) at In[36]:1
  myabs(!Matched::Float64) at In[32]:1

In [42]:
myabs(1 + 1im)

MethodError: MethodError: no method matching myabs(::Complex{Int64})
Closest candidates are:
  myabs(!Matched::Complex{Float64}) at In[36]:1
  myabs(!Matched::Float64) at In[32]:1

In [43]:
myabs(x::Real) = sign(x) * x
myabs(z::Complex) = sqrt(real(z * conj(z)))

myabs (generic function with 4 methods)

In [44]:
myabs(-3)

3

# Multiple Dispatch

Julia's dispatch mechanism always chooses the *most specific method* for the given input types.

In [45]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [46]:
methods(f)

# 5 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[45]:5
[2] f(a::Number, b::Number) in Main at In[45]:2
[3] f(a::Number, b) in Main at In[45]:3
[4] f(a, b::Number) in Main at In[45]:4
[5] f(a, b) in Main at In[45]:1

In [47]:
f(1.5, 2)

"a and b are both numbers"

In [48]:
f(1, "Köln!")

"a is a number"

In [49]:
f(1, 2)

"a and b are both integers"

In [50]:
f("Hello", "World!")

"fallback"

In [52]:
@which f(1, 2)

f(a::Integer, b::Integer) in Main at In[45]:5

In [53]:
@which f(1, "Köln!")

f(a::Number, b) in Main at In[45]:3

In [54]:
methods(+)

# 161 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:278
[2] +(x::Bool, y::Bool) in Base at bool.jl:96
[3] +(x::Bool) in Base at bool.jl:93
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:104
[5] +(x::Bool, z::Complex) in Base at complex.jl:285
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:279
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:293
[11] +(::Missing, ::Missing) in Base at missing.jl:93
[12] +(::Missing) in Base at missing.jl:79
[13] +(::Missing, ::Number) in Base at missing.jl:94
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:413
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:455
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:454
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:453
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:424
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt8}) in Base.GMP at gmp.jl:461
[21] +(x::BigInt, c::Union{Int16, Int32, Int8}) in Base.GMP at gmp.jl:467
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:563
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:556
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:550
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:409
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:378
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt8}) in Base.MPFR at mpfr.jl:385
[28] +(x::BigFloat, c::Union{Int16, Int32, Int8}) in Base.MPFR at mpfr.jl:393
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:401
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.2\Dates\src\periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.2\Dates\src\periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.2\Dates\src\periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.2\Dates\src\arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.2\Dates\src\arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.2\Dates\src\arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.2\Dates\src\arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.2\Dates\src\arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.2\Dates\src\arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.2\Dates\src\arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.2\Dates\src\arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\std

In [57]:
true + false

1

In [55]:
@which true + false

+(x::Bool, y::Bool) in Base at bool.jl:96

In [59]:
"Hello"*"World!"

"HelloWorld!"

In [58]:
@which "Hello"*"World!"

*(s1::Union{AbstractChar, AbstractString}, ss::Union{AbstractChar, AbstractString}...) in Base at strings/basic.jl:230

Julia's standard functions are not special by any means.

We can easily modify or add methods to them as well.

In [60]:
"Carsten" + "Bauer"

MethodError: MethodError: no method matching +(::String, ::String)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529

In [62]:
import Base: + # we have to import functions to override/extend them
+(x::String, y::String) = x * " " * y

+ (generic function with 162 methods)

In [ ]:
Base.:+(x::String, y::String) = 

In [63]:
"Kölle" + "Alaaf"

"Kölle Alaaf"

Any function based on the `+` operation can now handle `String`s as well.

In [64]:
sum(["This", "works", "although", "we", "never", "touched", "sum!"])

"This works although we never touched sum!"

It happens rarely, but it can happen that there is no unique most specific method:

In [65]:
f(x::Int64, y::Any) = println("int")
f(x::Any, y::String) = println("string")
f(3, "test")

MethodError: MethodError: f(::Int64, ::String) is ambiguous. Candidates:
  f(x, y::String) in Main at In[65]:2
  f(x::Int64, y) in Main at In[65]:1
Possible fix, define
  f(::Int64, ::String)

# Parametric types

Types can have *type parameters*. The most prominent example is Julia's array type.

In [66]:
M = rand(2,2)

2×2 Array{Float64,2}:
 0.552521  0.138808
 0.179865  0.521611

In [67]:
typeof(M)

Array{Float64,2}

Here, `Array` is a parametric array datatype. Its type parameters `Float64` and `2` indicate the type of the element the array can hold and and its dimensionality. Hence, we have a matrix of floating point numbers.

This generalizes as expected. Here, a matrix of `String`s:

In [68]:
M = fill("Cologne", 2,2)

2×2 Array{String,2}:
 "Cologne"  "Cologne"
 "Cologne"  "Cologne"

In [69]:
eltype(M)

String

In [70]:
M[1] = 3.2

MethodError: MethodError: Cannot `convert` an object of type Float64 to an object of type String
Closest candidates are:
  convert(::Type{T<:AbstractString}, !Matched::T<:AbstractString) where T<:AbstractString at strings/basic.jl:208
  convert(::Type{T<:AbstractString}, !Matched::AbstractString) where T<:AbstractString at strings/basic.jl:209
  convert(::Type{T}, !Matched::T) where T at essentials.jl:167

We can also nest parametric types. This is a vector of matrices of `Float64`s.

In [71]:
x = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [72]:
v = [rand(2,2) for i in 1:3]

3-element Array{Array{Float64,2},1}:
 [0.015380762921892721 0.22302545867845858; 0.04307206056465818 0.06920607359103448]
 [0.09790704137739459 0.629245749523311; 0.09825966559253851 0.9911695366793709]    
 [0.5254975723171518 0.015386619435769244; 0.46138332971899154 0.30563524253211605] 

In [73]:
eltype(v)

Array{Float64,2}

Since vectors and matrices pop up so frequently, Julia has a nice shortcut type alias for them.

In [74]:
3 == 3.0

true

In [75]:
3 === 3.0

false

In [79]:
a = [1,2,3]
b = a

3-element Array{Int64,1}:
 1
 2
 3

In [80]:
a === b

true

In [78]:
[1,2,3] === [1,2,3]

false

In [ ]:
Vector{Float64} === Array{Float64, 1}

In [ ]:
Matrix{Float64} === Array{Float64, 2}

Another example of a parametric type is the `Tuple`.

In [81]:
(1,2.0,"3")

(1, 2.0, "3")

In [82]:
typeof((1,2.0,"3"))

Tuple{Int64,Float64,String}

In [84]:
[1,2.0,"3"]

3-element Array{Any,1}:
 1   
 2.0 
  "3"

In [85]:
typeof([1,2.0,"3"])

Array{Any,1}

### `UnionAll` types and `where`

Note that parametric types have the following (somewhat counterintuitive at first) property

In [86]:
Vector{Float64} <: Vector{Real}

false

although we have

In [87]:
Float64 <: Real

true

The reason is that `Vector{Real}` is a concrete type - it describes a vector of values that individually have a type `T <: Real`  - and concrete types don't have subtypes.

In [88]:
isconcretetype(Vector{Real})

true

In [90]:
Real[1, 2.2, 13f0]

3-element Array{Real,1}:
  1    
  2.2  
 13.0f0

What we actually *mean* is

In [91]:
Vector{Float64} <: Vector{T} where T<:Real

true

Here, `Vector{T} where T <: Real` describes the *set* of concrete `Vector` types whose elements are of any single type `T` that is a subtype of `Real`.

In [92]:
Vector{Int64} <: Vector{T} where T<:Real

true

In [93]:
Vector{Real} <: Vector{T} where T<:Real

true

Using this notation, our `Vector{Real}` from above can more explicitly be written as `Vector{T where T<:Real}`

In [94]:
Vector{Real} === Vector{T where T<:Real}

true

### Type parameters in function signatures

In [96]:
h(x::Integer) = typeof(x)
# h(x::T) where T<:Integer = T

h (generic function with 1 method)

In [ ]:
h(x::T) where T<:Union{Float64, Int32}

In [ ]:
h(x::T) where T = T
# h(x) = typeof(x)

**Quick exercise**: Write a single-argument function that takes any real matrix as input and returns the element type of the matrix.

**Solution:**
<details>
  <summary>Click to reveal</summary>
<br>
    
```julia
g(x::Matrix{T}) where T<:Real = T
```

or alternatively

```julia
g(x::Matrix{<:Real}) = eltype(x)
```
</details>

**Test:**

In [98]:
g(x::Matrix{T}) where T<:Real = T

g (generic function with 1 method)

In [99]:
g(rand(Float32,2,2))

Float32

In [100]:
g(rand(Int16,2,2))

Int16

In [108]:
k(x::Matrix{Real}) = typeof(x)

k (generic function with 1 method)

In [103]:
k(rand(Float32,2,2))

MethodError: MethodError: no method matching k(::Array{Float32,2})
Closest candidates are:
  k(!Matched::Array{Real,2}) at In[102]:1

In [104]:
typeof(rand(Float32,2,2))

Array{Float32,2}

In [109]:
a = rand(Float32,2,2)

2×2 Array{Float32,2}:
 0.609444  0.518039 
 0.870425  0.0857042

In [110]:
a_real = Matrix{Real}(a)

2×2 Array{Real,2}:
 0.609444  0.518039 
 0.870425  0.0857042

In [111]:
k(a)

MethodError: MethodError: no method matching k(::Array{Float32,2})
Closest candidates are:
  k(!Matched::Array{Real,2}) at In[108]:1

In [112]:
k(a_real)

Array{Real,2}

# "Diagonal" dispatch

In [114]:
d(x::T, y::T) where T = "same type"
d(x, y) = "different types"

d (generic function with 2 methods)

In [115]:
d(3, 4)

"same type"

In [116]:
d(3.0, 1.0)

"same type"

In [117]:
d(1, 4.2)

"different types"

# Duck typing examples

### `UnitRange`

In [118]:
x = 1:30

1:30

In [120]:
collect(x)

30-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
  ⋮
 19
 20
 21
 22
 23
 24
 25
 26
 27
 28
 29
 30

In [121]:
typeof(x)

UnitRange{Int64}

In [122]:
typeof(x) <: AbstractArray

true

Because it is a subtype of `AbstractArray` we can do array-like things with it (it should basically behave like an array!)

In [125]:
x[3]

3

In [126]:
getindex(x, 3)

3

In [127]:
size(x)

(30,)

In [128]:
eltype(x)

Int64

However, it's not implemented like a regular `Array` at all.

In fact, it's just two numbers! We can see this by looking at it's fields:

In [129]:
fieldnames(typeof(x))

(:start, :stop)

or just by inspecting the source code

In [130]:
@which UnitRange(1, 30)

(::Type{UnitRange})(start::T, stop::T) where T<:Real in Base at range.jl:277

It is an `immutable` type which just holds the start and stop values.

This means that indexing, `A[i]`, is not just a look-up but a (small) function (try `@which getindex(x, 4)`).

What's nice about this is that we can use it in calculations and no array, containing the numbers from 1 to 30, is ever created.

Allocating memory is typically costly.

In [131]:
@time collect(1:10000000);

  0.036105 seconds (7 allocations: 76.294 MiB, 42.80% gc time)


But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [132]:
@time 1:10000000;

  0.000002 seconds (5 allocations: 192 bytes)


Yet, in code they *act* the same way.

# Other types

* Union types: `Union{Float64, Int32}`
* [Bitstypes](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/#man-bits-types-1) (check with `isbits(x)`, `isbitstype(T)`)
* [Value types](https://docs.julialang.org/en/v1/manual/types/#%22Value-types%22-1) (allows dispatch on values)

See https://docs.julialang.org/en/latest/manual/types/ for more.

# Extra: slurping and splatting

In [133]:
f(x...) = println(x) # slurping

f (generic function with 8 methods)

In [134]:
f(3, 1.2, "Carsten")

(3, 1.2, "Carsten")


In [135]:
f(1.2)

(1.2,)


In [136]:
f(1,2,3,4,5)

(1, 2, 3, 4, 5)


In [137]:
g(x::Vector) = +(x...) # splat vector into addition operation

g (generic function with 3 methods)

In [138]:
g([1,2,3])

6

# Core messages of this Notebook

* **Concrete types** describe data structures, i.e. concrete implementations.
* **Abstract types** define the kind of a thing (What is it? What can I do with it?), i.e. an informal interface. This is also known as **duck-typing**.
* A **function** (the what) can have multiple **methods** (the how).
* **Multiple dispatch**: Julia selects the method to run based on the types of all input arguments and chooses the most specialized one.
* Types can have parameters, i.e. `Vector{Float64}`. We can use the notation `T where T<:SomeSuperType` to address *sets* of types.